Aqui se explica el proceso para poder utilizar la red generativa entrenada en este proyecto

In [5]:

import os
import random
from PIL import Image,  ImageOps
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense
import time
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import copy
from sklearn.metrics import confusion_matrix
from torch.utils.data import RandomSampler, ConcatDataset
import pandas as pd
import shutil
from PIL import ImageFile
from tqdm import tqdm
from PIL import ImageEnhance


# Construir el modelo

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU no disponible, usando CPU")

Usando GPU: NVIDIA GeForce GTX 1060 6GB


In [7]:
class Generator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Generator, self).__init__()
        # Capas convolucionales
        self.conv1 = self.conv_block(in_channels, 32, kernel_size=4, stride=2)  # k4n32s2
        self.conv2 = self.conv_block(32, 64, kernel_size=4, stride=2)  # k4n64s2
        self.conv3 = self.conv_block(64, 128, kernel_size=4, stride=2)  # k4n128s2
        self.conv4 = self.conv_block(128, 256, kernel_size=4, stride=2)  # k4n256s2
        self.conv5 = self.conv_block(256, 256, kernel_size=4, stride=2)  # k4n256s2

        # Capas deconvolucionales con skip connections
        self.deconv6 = self.deconv_block(256, 256, kernel_size=4, stride=2)  # k4n256s2k3n256s1
        self.deconv7 = self.deconv_block(256*2, 128, kernel_size=4, stride=2)  # k4n128s2k3n128s1
        self.deconv8 = self.deconv_block(128*2, 64, kernel_size=4, stride=2)  # k4n64s2k3n64s1
        self.deconv9 = self.deconv_block(64*2, 32, kernel_size=4, stride=2)  # k4n32s2k3n32s1
        self.deconv10 = self.deconv_block(32*2, 32, kernel_size=4, stride=2)  # k4n32s2k3n32s1k3n3s1
        self.conv11 = nn.Conv2d(32, 3, 3, stride=1, padding=1, bias=False)
    def conv_block(self, in_channels, out_channels, kernel_size, stride):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def deconv_block(self, in_channels, out_channels, kernel_size, stride):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        conv4_out = self.conv4(conv3_out)
        conv5_out = self.conv5(conv4_out)

        deconv6_out = self.deconv6(conv5_out)
        deconv7_out = self.deconv7(torch.cat((conv4_out, deconv6_out), 1))  # Skip connection
        deconv8_out = self.deconv8(torch.cat((conv3_out, deconv7_out), 1))  # Skip connection
        deconv9_out = self.deconv9(torch.cat((conv2_out, deconv8_out), 1))  # Skip connection
        deconv10_out = self.deconv10(torch.cat((conv1_out, deconv9_out), 1))  # Skip connection
        conv11_out = self.conv11(deconv10_out)
        return torch.tanh(conv11_out)

# Cargar el peso

In [16]:
generator = Generator(3,3).to(device)

# Construir la ruta completa al archivo
PATH =f'..\Pesos\L2 + Angular + Gradiente + Reentreno\generator19.pth'

# Cargar los pesos del modelo desde el archivo
generator.load_state_dict(torch.load(PATH))


<All keys matched successfully>

# Uso de la red generativa con una imagen

In [17]:
transform = transforms.Compose([
    transforms.Resize((256,256)), #El primer componente es la altura de la imagen y el segundo es su ancho
    transforms.ToTensor() #Transforma las imagenes a tensor
])

In [18]:
ruta_imagen_prueba = '../ImagenPrueba/Generador/Input/n01496331_18538.JPG' # Se pone la ruta de la imagen que se desee modificar

# Cargar la imagen de prueba 
imagen_prueba = Image.open(ruta_imagen_prueba)
imagen_prueba = transform(imagen_prueba).unsqueeze(0) 
imagen_prueba = imagen_prueba.to(device)
with torch.no_grad():
    generator.eval()  # Poner el generador en modo de evaluación
    imagen_generada = generator(imagen_prueba)
nombre_archivo =f'..\\ExperimentoUnaImagen\\Output\\imagen_generada.JPG' #Guardar la imagen en la carpeta deseada, se recomienda que sea en ExperimentoUnaImagen 
torchvision.utils.save_image(imagen_generada, nombre_archivo)
print(f'Imagen de prueba guardada en {nombre_archivo}')

Imagen de prueba guardada en ..\ExperimentoUnaImagen\Output\imagen_generada.JPG
